Advanced Model

Preprocess

In [7]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# df_en = pd.read_csv("artifacts/preprocessed_comments.csv")
# df_en = pd.read_csv("artifacts/preprocessed_comments_all_eng.csv")
df_en = pd.read_csv("artifacts/preprocessed_comments_all_top10.csv")
# df_en = pd.read_csv("artifacts/preprocessed_comments_all_eng.csv")
print("Saved clean dataset:", df_en.shape)

Saved clean dataset: (1166207, 6)


In [ ]:
#segmentacja na zdania
import re

def simple_sentence_split(text: str):
    parts = re.split(r'(?<=[.!?])\s+|\n+', str(text))
    parts = [p.strip() for p in parts if p and p.strip()]
    return parts

df_en["sentences"] = df_en["text_clean"].apply(simple_sentence_split)
df_sent = df_en.explode("sentences").rename(columns={"sentences":"sentence"}).reset_index(drop=True)

# filtr ultra krótkich zdań
df_sent = df_sent[df_sent["sentence"].str.len() >= 10].reset_index(drop=True)
df_sent[["listing_id","sentence", "date"]].head(10), df_sent.shape


(   listing_id                                           sentence        date
 0        2737  this spot was in a great and nice area,walking...  2014-12-26
 1        2737     the room, that i rented, was clean and warm :)  2014-12-26
 2        2737  elif was really very helpful, while arranging ...  2015-01-04
 3        2737  the room was clean and cosy, with all the furn...  2015-01-04
 4        2737  picchioni was really helpful and helped me to ...  2015-01-04
 5        2737  elif's flat is spacious, clean, and had everyt...  2015-04-12
 6        2737  i did not get to meet elif, but she was patien...  2015-04-12
 7        2737  i met daniele, her husband, who clearly loves ...  2015-04-12
 8        2737  excellent hosts that made me want to live in i...  2015-04-12
 9        2737  i had a wonderful time staying with elif and h...  2015-04-22,
 (4885977, 7))

In [ ]:
df_doc = df_sent[["listing_id", "sentence", "date"]].copy()

df_doc["sentence"] = df_doc["sentence"].astype(str).str.strip()
df_doc = df_doc[df_doc["sentence"].str.len() >= 10].copy()
df_doc.shape, df_doc.head(5)


((4177443, 3),
    listing_id                                           sentence        date
 0        2737  this spot was in a great and nice area,walking...  2014-12-26
 1        2737     the room, that i rented, was clean and warm :)  2014-12-26
 2        2737  elif was really very helpful, while arranging ...  2015-01-04
 3        2737  the room was clean and cosy, with all the furn...  2015-01-04
 4        2737  picchioni was really helpful and helped me to ...  2015-01-04)

In [ ]:
# N_SENT = 200_000  # na start
# df_doc_small = df_doc.sample(n=min(N_SENT, len(df_doc)), random_state=42).reset_index(drop=True)

df_doc_small = df_doc

In [ ]:
import os
os.makedirs("artifacts", exist_ok=True)
df_doc_small.to_csv("artifacts/sentences_for_aspects_all_top10.csv", index=False)


Wektoryzacja

In [17]:
import pandas as pd

SAMPLE_PERCENT = 100 

df_doc_full = pd.read_csv("artifacts/sentences_for_aspects_all_top10.csv")
        
if SAMPLE_PERCENT < 100:
    n_sample = int(len(df_doc_full) * SAMPLE_PERCENT / 100)
    df_doc_small = df_doc_full.sample(n=n_sample, random_state=42).reset_index(drop=True)
    print(f"Used {SAMPLE_PERCENT}% of data: {len(df_doc_small)} out of {len(df_doc_full)} rows")
else:
    df_doc_small = df_doc_full
    print(f"Used all data: {len(df_doc_small)} rows")

Used all data: 5994757 rows


In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

from sentence_transformers import SentenceTransformer
emb_model = SentenceTransformer(        
    # "paraphrase-multilingual-MiniLM-L12-v2",
    # "all-mpnet-base-v2", # for eng data
    "intfloat/multilingual-e5-base", # for top10 lang
    device="cuda"
)
emb_model = emb_model.half()
emb_model.max_seq_length = 128

emb_model.get_sentence_embedding_dimension()

2.5.1+cu118
11.8
True
Quadro M2200


2.5.1+cu118
11.8
True
Quadro M2200


c:\EITI\NOW\ium\.venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2.5.1+cu118
11.8
True
Quadro M2200


c:\EITI\NOW\ium\.venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


768

In [14]:
sentences = df_doc_small["sentence"].tolist()
len(sentences)


2997378

In [ ]:
embeddings = emb_model.encode(
    sentences,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)


Batches:   0%|          | 0/46835 [00:00<?, ?it/s]

In [16]:
import numpy as np

print(type(embeddings))
print(embeddings.shape)
print("NaN count:", np.isnan(embeddings).sum())


<class 'numpy.ndarray'>
(2997378, 768)
NaN count: 0


In [ ]:
import os
os.makedirs("artifacts", exist_ok=True)

# np.save("artifacts/sentence_embeddings.npy", embeddings)
# np.save("artifacts/sentence_embeddings_all_eng_30.npy", embeddings)
# np.save("artifacts/sentence_embeddings_all_top10_30.npy", embeddings)
np.save("artifacts/sentence_embeddings_all_top10.npy", embeddings)

# df_doc_small.to_csv("artifacts/sentences_for_aspects_en.csv", index=False)
df_doc_small.to_csv("artifacts/sentences_for_aspects_all_top10.csv", index=False)
print("Saved embeddings:", embeddings.shape)


Saved embeddings: (2997378, 768)


Grupowanie

KMENAS (MINIBATCH) - ostateczny model

In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from tqdm.auto import tqdm
import os

K = 22
TEST_SAMPLE = None
BATCH_SIZE = 5000
SAMPLE_FOR_METRICS = 100000

emb_path = "artifacts/sentence_embeddings_all_top10_100.npy"
sent_path = "artifacts/sentences_for_aspects_all_top10_100.csv"

embeddings_mmap = np.load(emb_path, mmap_mode='r')
df_full = pd.read_csv(sent_path)

if TEST_SAMPLE is not None and TEST_SAMPLE < len(df_full):
    indices = np.random.choice(len(df_full), TEST_SAMPLE, replace=False)
    embeddings = embeddings_mmap[indices]
    df_doc_small = df_full.iloc[indices].reset_index(drop=True)
    print(f"Test mode: {TEST_SAMPLE} samples")
else:
    embeddings = embeddings_mmap
    df_doc_small = df_full
    print(f"Full dataset: {len(df_doc_small)} rows")

print(f"Shape: {embeddings.shape}, dtype: {embeddings.dtype}")

total_start = time.time()

kmeans = MiniBatchKMeans(
    n_clusters=K, 
    random_state=42, 
    batch_size=BATCH_SIZE,
    n_init=10,
    max_iter=300,
    verbose=0,
    reassignment_ratio=0.01
)

print("Training")
n_batches = int(np.ceil(len(embeddings) / BATCH_SIZE))
for i in range(5):
    for batch_idx in range(n_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = min(start_idx + BATCH_SIZE, len(embeddings))
        batch = embeddings[start_idx:end_idx]
        kmeans.partial_fit(batch)
    print(f"Epoch {i+1}/5")

print("Predicting labels")
labels_kmeans = np.zeros(len(embeddings), dtype=np.int32)
for batch_idx in range(n_batches):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min(start_idx + BATCH_SIZE, len(embeddings))
    batch = embeddings[start_idx:end_idx]
    labels_kmeans[start_idx:end_idx] = kmeans.predict(batch)

df_doc_small["cluster_kmeans"] = labels_kmeans

print("Calculating metrics")
if len(embeddings) > SAMPLE_FOR_METRICS:
    sample_idx = np.random.choice(len(embeddings), SAMPLE_FOR_METRICS, replace=False)
    emb_sample = embeddings[sample_idx]
    labels_sample = labels_kmeans[sample_idx]
else:
    emb_sample = embeddings
    labels_sample = labels_kmeans

sil_kmeans = silhouette_score(emb_sample, labels_sample)
db_kmeans = davies_bouldin_score(emb_sample, labels_sample)
ch_kmeans = calinski_harabasz_score(emb_sample, labels_sample)

total_time = time.time() - total_start

print(f"\nResults:")
print(f"Silhouette: {sil_kmeans:.4f}")
print(f"Davies-Bouldin: {db_kmeans:.4f}")
print(f"Calinski-Harabasz: {ch_kmeans:.2f}")
print(f"Time: {total_time/60:.1f} min")
print(f"Cluster sizes: min={pd.Series(labels_kmeans).value_counts().min()}, max={pd.Series(labels_kmeans).value_counts().max()}")

Full dataset: 5994757 rows
Shape: (5994757, 768), dtype: float16
Training
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Predicting labels
Calculating metrics

Results:
Silhouette: 0.0194
Davies-Bouldin: 4.5635
Calinski-Harabasz: 993.10
Time: 11.6 min
Cluster sizes: min=184375, max=431306


In [3]:
for c in range(K):
    print(f"\nK-MEANS CLUSTER {c}")
    examples = df_doc_small[df_doc_small["cluster_kmeans"] == c]["sentence"].sample(15, random_state=1)
    for s in examples:
        print("-", s)


K-MEANS CLUSTER 0
- if you ha e plans to go to rome, don't doubt to book this place.
- staying in a piece of history while seeing all the roman history is truly magical.
- maria was very nice and gave us tips for our stay in rome.
- best stay in rome!100% recommendation!
- we enjoyed rome fully!
- sin duda, recomendamos este alojamiento a cualquiera que desee vivir una experiencia auténtica en roma.<br/>¡gracias, vale!
- paolo’s the man and his flat in rome is the perfect place to take in the city.
- we will definitely book again our next trip to rome!
- <br/>the location is great, we easily got to rome, the beach, and a vineyard, so had a full and enjoyable weekend despite the shortfalls of the hosts.
- i would definitely would share this apartment with anyone i knew was going to go to rome.
- we loved this place even though it was in the ‘most happening’ place in trastevere.<br/>we were family of 4 from boston visiting rome, napoli and florence.
- if you’re looking for a place in ro

In [ ]:
cluster_merge_map = {
    0: 0,
    1: 1,
    2: 2,
    5: 3,
    6: 4,
    7: 5,
    8: 6,
    9: 7,
    10: 8,
    11: 9,
    12: 10,
    13: 11,
    14: 12,
    15: 13,
    16: 14,
    17: 15,
    19: 16,
    20: 17,
    21: 18,
    3: 19,
    4: -1,
    18: -1,
}

df_doc_small["cluster_merged"] = df_doc_small["cluster_kmeans"].map(cluster_merge_map)
print("Merged clusters")
print(df_doc_small["cluster_merged"].value_counts().sort_index())
print(f"\nRemoved clusters: {(df_doc_small['cluster_merged'] == -1).sum()}")
print(f"Percent removed: {(df_doc_small['cluster_merged'] == -1).mean():.2%}")

Merged clusters
cluster_merged
-1     494755
 0     400863
 1     212412
 2     217713
 3     264503
 4     377472
 5     431306
 6     229221
 7     307393
 8     299279
 9     210925
 10    366724
 11    205926
 12    254765
 13    252695
 14    271181
 15    273130
 16    189382
 17    184375
 18    313404
 19    237333
Name: count, dtype: int64
\Removed clusters: 494755
Percent removed: 8.25%


In [19]:
df_doc_small.sample(10)


,listing_id,sentence,date,cluster_kmeans,cluster_merged
2657077,1274571879866174539,posizione ottima a pochi minuti dalla metro ci...,2024-11-03,11,9
5034983,39714516,nicoletta and her staff were very helpful and ...,2023-07-02,1,1
468493,16460416,<br/>would highly recommend this apartment to ...,2023-10-21,0,0
2209947,4534022,"the apartment was clean, quite and near to the...",2017-04-30,1,1
2595808,37832975,federica and mauro are the best nocarz hosts i...,2021-12-24,6,4
593150,19859481,a couple blocks away and you are there.,2017-11-23,19,16
572898,3028770,"<br/>la casa è curata, pulita e completa di tu...",2015-01-03,9,7
5170717,47472227,thank you for sharing your place with my family!,2024-09-28,13,11
4145325,13182399,i definitely sat down and played the piano a f...,2018-03-27,17,15
5969113,1214233886038179033,we didn’t hear any more and so it wasn’t able ...,2024-11-09,8,6


In [ ]:
import os

os.makedirs("artifacts/kmeans", exist_ok=True)
df_doc_small.to_csv(
    "artifacts/kmeans/sentences_with_kmeans_clusters_top10_all_final.csv",
    index=False
)
print(f"Saved {len(df_doc_small)} sentences")


Saved 5994757 sentences


['artifacts/kmeans/kmeans_model_all_top10_all_final.joblib']

Poniżej aż do "Analiza sentymentu zdań" część z testowanych setupow, które nie sa ostatecznym modelem

jeden z setupow testow grupowania: clustering i BERTOPIC - nie uzyte na koncu

In [7]:
import pandas as pd
import numpy as np
import time
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from tqdm.auto import tqdm
import os

K = 22
TEST_SAMPLE = None
TEST_SAMPLE = 100000

emb_path = "artifacts/sentence_embeddings_all_eng_30.npy"
# emb_path = "artifacts/sentence_embeddings_all_top10_30.npy"
sent_path = "artifacts/sentences_for_aspects_all_eng_30.csv"
# sent_path = "artifacts/sentences_for_aspects_all_top10_30.csv"

embeddings_full = np.load(emb_path)
df_full = pd.read_csv(sent_path)

if TEST_SAMPLE is not None and TEST_SAMPLE < len(df_full):
    indices = np.random.choice(len(df_full), TEST_SAMPLE, replace=False)
    embeddings = embeddings_full[indices]
    df_doc_small = df_full.iloc[indices].reset_index(drop=True)
    print(f"Test mode: {TEST_SAMPLE} samples")
else:
    embeddings = embeddings_full
    df_doc_small = df_full
    print(f"Full dataset: {len(df_doc_small)} rows")

print(f"Embeddings: {embeddings.shape}")
print(f"Sentences: {df_doc_small.shape}")

total_start = time.time()

print("\nK-MEANS")
start = time.time()
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10, verbose=0)
labels_kmeans = kmeans.fit_predict(embeddings)
time_kmeans = time.time() - start
print(f"Training: {time_kmeans:.2f}s")

start_metrics = time.time()
sil_kmeans = silhouette_score(embeddings, labels_kmeans)
db_kmeans = davies_bouldin_score(embeddings, labels_kmeans)
ch_kmeans = calinski_harabasz_score(embeddings, labels_kmeans)
metrics_time = time.time() - start_metrics

print(f"Silhouette: {sil_kmeans:.4f}")
print(f"Davies-Bouldin: {db_kmeans:.4f}")
print(f"Calinski-Harabasz: {ch_kmeans:.2f}")
print(f"Total time: {time_kmeans + metrics_time:.2f}s")

df_doc_small["cluster_kmeans"] = labels_kmeans

print("\nBERTOPIC")
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

start = time.time()

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=50, min_samples=10, metric='euclidean', prediction_data=True)
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english', min_df=5, max_df=0.5)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    min_topic_size=30,
    nr_topics=K,
    calculate_probabilities=False,
    verbose=True
)

sentences = df_doc_small["sentence"].tolist()
labels_bertopic, _ = topic_model.fit_transform(sentences, embeddings)

mask = np.array(labels_bertopic) != -1
if mask.sum() > 0:
    sil_bert = silhouette_score(embeddings[mask], np.array(labels_bertopic)[mask])
    db_bert = davies_bouldin_score(embeddings[mask], np.array(labels_bertopic)[mask])
    ch_bert = calinski_harabasz_score(embeddings[mask], np.array(labels_bertopic)[mask])
    noise = 1 - mask.mean()
else:
    sil_bert = db_bert = ch_bert = noise = float('nan')

time_bertopic = time.time() - start

print(f"Noise ratio: {noise:.2%}")
print(f"Silhouette: {sil_bert:.4f}")
print(f"Davies-Bouldin: {db_bert:.4f}")
print(f"Calinski-Harabasz: {ch_bert:.2f}")
print(f"Total time: {time_bertopic:.2f}s")

df_doc_small["cluster_bertopic"] = labels_bertopic

total_time = time.time() - total_start
print(f"\nTotal experiment time: {total_time/60:.2f} min")

comparison = pd.DataFrame([
    {"Method": "K-Means", "Time(s)": time_kmeans + metrics_time, "Silhouette": sil_kmeans, "Davies-Bouldin": db_kmeans, "Calinski-Harabasz": ch_kmeans},
    {"Method": "BERTopic", "Time(s)": time_bertopic, "Silhouette": sil_bert, "Davies-Bouldin": db_bert, "Calinski-Harabasz": ch_bert}
])
print("\n", comparison.round(4))

Test mode: 100000 samples
Embeddings: (100000, 768)
Sentences: (100000, 2)

K-MEANS
Training: 108.72s
Silhouette: 0.0360
Davies-Bouldin: 3.4736
Calinski-Harabasz: 1579.70
Total time: 444.48s

BERTOPIC


2026-01-17 17:33:16,261 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-17 17:36:57,455 - BERTopic - Dimensionality - Completed ✓
2026-01-17 17:36:57,459 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-17 17:37:07,958 - BERTopic - Cluster - Completed ✓
2026-01-17 17:37:07,959 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-01-17 17:37:10,615 - BERTopic - Representation - Completed ✓
2026-01-17 17:37:10,615 - BERTopic - Topic reduction - Reducing number of topics
2026-01-17 17:37:10,949 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-17 17:37:13,062 - BERTopic - Representation - Completed ✓
2026-01-17 17:37:13,078 - BERTopic - Topic reduction - Reduced number of topics from 218 to 22


Noise ratio: 39.13%
Silhouette: 0.0389
Davies-Bouldin: 3.2504
Calinski-Harabasz: 515.64
Total time: 348.51s

Total experiment time: 13.84 min

      Method   Time(s)  Silhouette  Davies-Bouldin  Calinski-Harabasz
0   K-Means  444.4836      0.0360          3.4736          1579.6980
1  BERTopic  348.5054      0.0389          3.2504           515.6375


In [ ]:
for c in range(min(5, K)):
    print(f"\nK-MEANS CLUSTER {c}")
    examples = df_doc_small[df_doc_small["cluster_kmeans"] == c]["sentence"].sample(10, random_state=1)
    for s in examples:
        print("-", s)

print("\nBERTopic top words:")
for topic_id in range(min(5, len(topic_model.get_topics()))):
    if topic_id == -1:
        continue
    words = topic_model.get_topic(topic_id)
    if words:
        print(f"Topic {topic_id}: {', '.join([w for w, _ in words[:8]])}")


=== CLUSTER 0 ===
- alojamiento limpio, amplio y bien equipado.<br/>dada su cercanía con el transporte público, es adecuado para llegar y conocer todos los puntos de interés que tiene la ciudad.<br/>ideal para el buen descanso.
- sotto l’appartamento ci sono vari locali come ristoranti o bar per la colazione.
- non manca davvero niente.<br/>anche se sono rimasto una sola notte, sarei rimasto davvero di più, anche perchè la posizione è ideale per raggiungere molte delle maggiori attrazioni della città.
- ottima posizione (parcheggio, ingresso musei vaticani, metro, ristoranti)<br/>buon appartamento, spazioso, pulito
- appartement idéalement situé pour découvrir rome à pied <br/>appartement idéal pour 6 personnes ( 4 adultes et 2 enfants ) <br/>organisation parfaite gérée par claudia pour venir nous chercher à l'aéroport et nous ramener à l'appartement<br/>et rome quelle ville magnifique <br/>
- il b&b di fabiana si trova in una posizione centrale,vicino alla fermata di autobus che port

In [ ]:
cluster_to_aspect = {
    0: "location_convenience",
    1: "overall_recommendation",
    2: "host_communication",
    3: "overall_experience",
    4: "amenities_facilities",
    5: "space_quietness",
    6: "apartment_cleanliness",
    7: "generic_positive",
    8: "host_recommendations",
    9: "transport_walking",
    10: "host_hospitality",
    11: "problems_issues",
    12: "restaurants_bars",
    13: "apartment_aesthetics",
    14: "rome_experience",
    15: "host_gestures",
    16: "location_generic",
}

df_doc_small["aspect_kmeans"] = df_doc_small["cluster_merged"].map(cluster_to_aspect)
print(f"Liczba zdań: {len(df_doc_small)}")
print(f"\nAspekty:")
print(df_doc_small["aspect_kmeans"].value_counts())

Liczba zdań: 1798427

Aspekty:
aspect_kmeans
overall_recommendation    243821
apartment_cleanliness     165656
transport_walking         160348
host_communication        144974
problems_issues           122663
location_convenience      113847
generic_positive          108402
apartment_aesthetics       89874
amenities_facilities       84384
rome_experience            83717
host_gestures              83230
location_generic           82507
host_hospitality           71352
overall_experience         71013
restaurants_bars           64685
space_quietness            61867
host_recommendations       46087
Name: count, dtype: int64


In [ ]:
import os

os.makedirs("artifacts/kmeans", exist_ok=True)

df_doc_small.to_csv(
    "artifacts/kmeans/sentences_with_kmeans_clusters_allen_all_top10_30.csv",
    index=False
)
print(f"Zapisano {len(df_doc_small)} zdań")

In [ ]:
import joblib
joblib.dump(kmeans, "artifacts/kmeans/kmeans_model_all_top10_30.joblib")


['artifacts/kmeans/kmeans_model_all_top10_30.joblib']

jeden z setupow testsow grupowania: HDBSCAN - nie uzyte na koncu


In [1]:
import pandas as pd
import numpy as np
df_doc_small = pd.read_csv("artifacts/kmeans/sentences_with_kmeans_clusters.csv")
embeddings = np.load("artifacts/sentence_embeddings.npy")
print("Loaded embeddings:", embeddings.shape)

Loaded embeddings: (200000, 384)


In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=100, 
    min_samples=10,
    metric="euclidean",
    cluster_selection_method="eom",
    core_dist_n_jobs=-1,
    
)
labels_hdb = clusterer.fit_predict(embeddings)
labels_hdb[:20]


array([ 0,  0, -1,  0,  0,  0,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1,  0,
       -1, -1,  0])

In [ ]:
df_doc_small["cluster_hdbscan"] = labels_hdb
df_doc_small["cluster_hdbscan"].value_counts().head(10)


cluster_hdbscan
-1    102264
 0     97625
 1       111
Name: count, dtype: int64

In [ ]:
n_clusters = len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0)
noise_ratio = (labels_hdb == -1).mean()
n_clusters, noise_ratio


(2, np.float64(0.51132))

In [5]:
for c in sorted(df_doc_small["cluster_hdbscan"].unique()):
    if c == -1:
        continue
    print(f"\n=== HDBSCAN CLUSTER {c} ===")
    examples = df_doc_small[df_doc_small["cluster_hdbscan"] == c]["sentence"].sample(8, random_state=1)
    for s in examples:
        print("-", s)



=== HDBSCAN CLUSTER 0 ===
- the neighborhood seemed safe.
- 10/10 recommend.
- we are very glad we stayed here and would recommend it to others!
- andrea was very nice and helpful!
- he is so kind and helpful!
- location is excellent to walk to sights and the neighbourhood has fabulous bars and restaurants but it is generally quiet and relaxing.
- livia is an amazing host!
- really close to vatican and accessible to other attractions.

=== HDBSCAN CLUSTER 1 ===
- i wish i could give this listing more than 5 stars!
- five stars from me!!
- all in all 5 stars!
- 5 stars all the way!!!
- i would recommend to anyone with all 5 stars!
- 5 stars ⭐️
- 6 stars no less!
- overall if we could give 6 stars we would!


In [24]:
CLUSTER_TO_ASPECT = {
    0: "location_centrality",
    1: "apartment_cleanliness_comfort",
    2: "host_helpfulness",
    3: "transport_access",
    4: "return_intention",
    5: "checkin_communication",
    6: "public_transport_proximity",
    7: "recommendation",
    8: "restaurants_shops",
    9: "recommendation_strong",
    10: "return_visit_intent",
    11: "apartment_quality_location",
    12: "stay_recommendation",
    13: "overall_experience",
    14: "host_quality",
    15: "cleanliness_location",
    16: "stay_recommendation_generic",
    17: "apartment_quality",
    18: "bed_comfort"
}


In [25]:
df_doc_small["aspect_hdbscan"] = df_doc_small["cluster_hdbscan"].map(CLUSTER_TO_ASPECT)
df_doc_small["aspect_hdbscan"].notna().mean()
df_doc_small["aspect_hdbscan"].value_counts().head(10)


aspect_hdbscan
apartment_quality             514
host_quality                  132
stay_recommendation           121
cleanliness_location           96
restaurants_shops              95
recommendation                 76
host_helpfulness               73
overall_experience             65
return_visit_intent            51
public_transport_proximity     50
Name: count, dtype: int64

In [26]:
df_doc_small[["sentence","aspect_hdbscan"]].sample(10)

,sentence,aspect_hdbscan
8734,"also, it would have been helpful to have extra...",NaN
16338,we loved the location and the beautiful privat...,NaN
9419,"his hospitality, responsiveness and overall he...",NaN
12451,trastevere is a hip little section in rome wit...,NaN
10235,it's located in nice neighborhood and you can ...,NaN
12111,could not ask for a more wonderful host!!!!,NaN
6990,we noticed the wifi was not working when we dr...,NaN
13943,diverse neighborhood.,NaN
8404,the apartment is convenient for two person wit...,NaN
16020,really nice and clean flat!,NaN


In [ ]:
os.makedirs("artifacts/hdbscan", exist_ok=True)
df_doc_small.to_csv("artifacts/hdbscan/sentences_with_hdbscan.csv", index=False)
print("HDBSCAN artifacts saved")


HDBSCAN artifacts saved


Analiza sentymentu zdań

In [25]:
from google.protobuf import message
from transformers import pipeline
import torch

print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-xlm-roberta-base-sentiment",
    device=0,
    truncation=True,
    max_length=512
)


True Quadro M2200


c:\EITI\NOW\ium\.venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
import pandas as pd
# df_doc_small = pd.read_csv("artifacts/kmeans/sentences_with_kmeans_clusters_allen_all_top10_30.csv")
# df_doc_small = pd.read_csv("artifacts/kmeans/sentences_with_kmeans_clusters_top10_all.csv")
df_doc_small = pd.read_csv("artifacts/kmeans/sentences_with_kmeans_clusters_top10_all_final.csv")



In [27]:
df_doc_small.sample(5)

,listing_id,sentence,date,cluster_kmeans,cluster_merged
2188017,16346946,small but the terrace is fantastic!,2024-09-27,16,14
114757,1096134326587113539,"barrio con mucho ambiente: restaurantes, cafet...",2024-05-05,5,3
3215857,1189026,it was so convenient for my wife & i.,2023-02-05,21,18
2542920,41682195,perfect for a family of 4 or 5.,2024-06-19,3,-1
2955690,768667628250951907,el alojamiento es excelente lindo barrio muy b...,2024-05-10,12,10


In [28]:
import numpy as np
from tqdm import tqdm

def batch_sentiment(texts, batch_size=32):
    results = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Sentiment analysis"):
        batch = texts[i:i+batch_size]
        out = sentiment_pipe(batch)
        results.extend(out)
    return results

sent_results = batch_sentiment(df_doc_small["sentence"].tolist(), batch_size=32)

Sentiment analysis: 100%|██████████| 187337/187337 [62:35:15<00:00,  1.20s/it]   


In [29]:
def normalize_sentiment_label(label):
    if label in ["positive", "neutral", "negative"]:
        return label
    return {
        "LABEL_0": "negative",
        "LABEL_1": "neutral",
        "LABEL_2": "positive"
    }[label]

df_doc_small["sentiment"] = [normalize_sentiment_label(r["label"]) for r in sent_results]

In [30]:
df_doc_small["sentiment"].value_counts(normalize=True)

sentiment
positive    0.750749
neutral     0.169533
negative    0.079718
Name: proportion, dtype: float64

In [ ]:
df_doc_small.to_csv("artifacts/sentences_with_sentiment_all_top10_all_final.csv", index=False)
df_doc_small.head()


,listing_id,sentence,date,cluster_kmeans,cluster_merged,sentiment
0,27753465,excellent séjour chez un hôte qui prépare votr...,2021-06-27,10,8,positive
1,31469836,"wifi works perfectly, it's spacious, stylish, ...",2020-09-24,12,10,positive
2,14759035,excelente relación precio-calidad por lo bien ...,2023-04-14,12,10,positive
3,722580592391061512,"great accommodation, has everything you need t...",2024-05-23,16,14,positive
4,14113698,"mais bon, proche du transport.",2017-04-01,19,16,neutral


przyklad dla only english - nie uzyte na koncu

In [ ]:
from google.protobuf import message
from transformers import pipeline
import torch

print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=0,
    truncation=True,
    max_length=512
)


True Quadro M2200


c:\EITI\NOW\ium\.venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

c:\EITI\NOW\ium\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\piotr\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to r

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
import pandas as pd
df_doc_small = pd.read_csv("artifacts/kmeans/sentences_with_kmeans_clusters_allen_all_eng_30.csv")

In [ ]:
import numpy as np
from tqdm import tqdm

def batch_sentiment(texts, batch_size=32):
    results = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Sentiment analysis"):
        batch = texts[i:i+batch_size]
        out = sentiment_pipe(batch)
        results.extend(out)
    return results

sent_results = batch_sentiment(df_doc_small["sentence"].tolist(), batch_size=32)

Sentiment analysis: 100%|██████████| 38695/38695 [13:48:18<00:00,  1.28s/it]  


In [10]:
def normalize_sentiment_label(label):
    if label in ["positive", "neutral", "negative"]:
        return label
    return {
        "LABEL_0": "negative",
        "LABEL_1": "neutral",
        "LABEL_2": "positive"
    }[label]

df_doc_small["sentiment"] = [normalize_sentiment_label(r["label"]) for r in sent_results]

In [11]:
df_doc_small["sentiment"].value_counts(normalize=True)

sentiment
positive    0.764902
neutral     0.190909
negative    0.044189
Name: proportion, dtype: float64

In [ ]:
df_doc_small.to_csv("artifacts/sentences_with_sentiment_all_eng_30.csv", index=False)
df_doc_small.head()


,listing_id,sentence,cluster_kmeans,cluster_merged,aspect_kmeans,sentiment
0,852674857517261824,we felt comfortable regardless.<br/>there are ...,19,12,restaurants_shops,positive
1,764132838081751517,"however, this place is not for tall people!",6,5,problems_issues,negative
2,20160040,"alessandra had a full refrigerator with water,...",7,6,kitchen_amenities,positive
3,2087982,the host had made it cosy and it was tidy and ...,12,7,apartment_quality,positive
4,31461596,great host and home for our stay.,11,0,overall_recommendation,positive


PIVOT + zmiana ostatecznych nazw (po testach)

In [8]:
import pandas as pd
df_sentences_eng = pd.read_csv("artifacts/sentences_with_sentiment_all_top10_all_final.csv")
df_sentences_eng.head()

,listing_id,sentence,date,cluster_kmeans,cluster_merged,sentiment
0,27753465,excellent séjour chez un hôte qui prépare votr...,2021-06-27,10,8,positive
1,31469836,"wifi works perfectly, it's spacious, stylish, ...",2020-09-24,12,10,positive
2,14759035,excelente relación precio-calidad por lo bien ...,2023-04-14,12,10,positive
3,722580592391061512,"great accommodation, has everything you need t...",2024-05-23,16,14,positive
4,14113698,"mais bon, proche du transport.",2017-04-01,19,16,neutral


In [ ]:
cluster_to_aspect_final = {
    0: "overall_positive",
    7: "overall_positive",
    10: "overall_positive",
    11: "overall_positive",
    18: "overall_positive",
    
    1: "apartment_quality",
    13: "apartment_quality",
    4: "host_communication",
    8: "host_communication",
    9: "transport_proximity",
    16: "transport_proximity",
    2: "space_location_features",
    3: "neighborhood_services", 
    5: "host_arrival_services",
    6: "equipment_supplies",
    12: "environmental_conditions",
    14: "location_quality",
    15: "apartment_amenities",
    17: "infrastructure_accessibility",
}

df_sentences_eng["aspect"] = df_sentences_eng["cluster_merged"].map(cluster_to_aspect_final)
df_sentences_eng_filtered = df_sentences_eng.dropna(subset=["aspect"]).copy()

print(f"Przed filtrowaniem: {len(df_sentences_eng)} zdań")
print(f"Po filtrowaniu: {len(df_sentences_eng_filtered)} zdań")
print(f"\nRozkład aspektów:")
print(df_sentences_eng_filtered["aspect"].value_counts())

df_sentences_eng_filtered.to_csv("artifacts/sentences_with_sentiment_all_top10_all_final_WITH_ASPECTS2.csv", index=False)

df_sentences_eng_filtered.head()

Przed filtrowaniem: 5994757 zdań
Po filtrowaniu: 5262669 zdań

Rozkład aspektów:
aspect
overall_positive                1594310
host_communication               676751
apartment_quality                465107
host_arrival_services            431306
transport_proximity              400307
apartment_amenities              273130
location_quality                 271181
neighborhood_services            264503
environmental_conditions         254765
equipment_supplies               229221
space_location_features          217713
infrastructure_accessibility     184375
Name: count, dtype: int64


,listing_id,sentence,date,cluster_kmeans,cluster_merged,sentiment,aspect
0,27753465,excellent séjour chez un hôte qui prépare votr...,2021-06-27,10,8,positive,host_communication
1,31469836,"wifi works perfectly, it's spacious, stylish, ...",2020-09-24,12,10,positive,overall_positive
2,14759035,excelente relación precio-calidad por lo bien ...,2023-04-14,12,10,positive,overall_positive
3,722580592391061512,"great accommodation, has everything you need t...",2024-05-23,16,14,positive,location_quality
4,14113698,"mais bon, proche du transport.",2017-04-01,19,16,neutral,transport_proximity


In [10]:
baseline_agg = (
    df_sentences_eng_filtered
    .dropna(subset=["aspect"])
    .groupby(["listing_id", "aspect", "sentiment", "date"])
    .size()
    .reset_index(name="count")
)

print(f"Baseline agregacja: {baseline_agg.shape}")
baseline_agg.head(10)

Baseline agregacja: (4394999, 5)


,listing_id,aspect,sentiment,date,count
0,2737,apartment_amenities,positive,2015-04-22,1
1,2737,apartment_quality,positive,2014-12-26,1
2,2737,apartment_quality,positive,2015-04-22,1
3,2737,host_arrival_services,positive,2015-01-04,2
4,2737,host_arrival_services,positive,2015-04-12,2
5,2737,host_communication,positive,2015-04-12,1
6,2737,host_communication,positive,2015-05-08,1
7,2737,overall_positive,positive,2015-01-04,1
8,2737,overall_positive,positive,2015-04-12,1
9,2737,overall_positive,positive,2015-04-22,3


In [11]:
baseline_pivot = (
    baseline_agg
    .pivot_table(
        index=["listing_id", "aspect", "date"],
        columns="sentiment",
        values="count",
        fill_value=0
    )
    .reset_index()
)

for col in ["positive", "neutral", "negative"]:
    if col not in baseline_pivot.columns:
        baseline_pivot[col] = 0

baseline_pivot["score"] = (
    baseline_pivot["positive"]
    - baseline_pivot["negative"]
)

baseline_pivot["total_mentions"] = (
    baseline_pivot["positive"]
    + baseline_pivot["neutral"]
    + baseline_pivot["negative"]
)

print(f"Baseline pivot: {baseline_pivot.shape}")
baseline_pivot.head(10)

Baseline pivot: (4116325, 8)


sentiment,listing_id,aspect,date,negative,neutral,positive,score,total_mentions
0,2737,apartment_amenities,2015-04-22,0.0,0.0,1.0,1.0,1.0
1,2737,apartment_quality,2014-12-26,0.0,0.0,1.0,1.0,1.0
2,2737,apartment_quality,2015-04-22,0.0,0.0,1.0,1.0,1.0
3,2737,host_arrival_services,2015-01-04,0.0,0.0,2.0,2.0,2.0
4,2737,host_arrival_services,2015-04-12,0.0,0.0,2.0,2.0,2.0
5,2737,host_communication,2015-04-12,0.0,0.0,1.0,1.0,1.0
6,2737,host_communication,2015-05-08,0.0,0.0,1.0,1.0,1.0
7,2737,overall_positive,2015-01-04,0.0,0.0,1.0,1.0,1.0
8,2737,overall_positive,2015-04-12,0.0,0.0,1.0,1.0,1.0
9,2737,overall_positive,2015-04-22,0.0,0.0,3.0,3.0,3.0


In [12]:
def ranking_for_listing(df, listing_id, top_k=3):
    sub = df[df["listing_id"] == listing_id]
    return (
        sub.sort_values("score", ascending=False).head(top_k),
        sub.sort_values("score", ascending=True).head(top_k)
    )

In [13]:
listing = df_sentences_eng_filtered["listing_id"].sample(1).iloc[0]

ranking_for_listing(baseline_pivot, listing)

(sentiment  listing_id              aspect        date  negative  neutral  \
 2335813      26705768    overall_positive  2024-10-24       0.0      0.0   
 2335691      26705768    overall_positive  2018-08-12       0.0      1.0   
 2335570      26705768  host_communication  2024-02-01       0.0      3.0   
 
 sentiment  positive  score  total_mentions  
 2335813         4.0    4.0             4.0  
 2335691         4.0    4.0             5.0  
 2335570         4.0    4.0             7.0  ,
 sentiment  listing_id                        aspect        date  negative  \
 2335511      26705768            equipment_supplies  2020-01-21       2.0   
 2335516      26705768            equipment_supplies  2023-07-01       2.0   
 2335582      26705768  infrastructure_accessibility  2019-07-20       1.0   
 
 sentiment  neutral  positive  score  total_mentions  
 2335511        0.0       0.0   -2.0             2.0  
 2335516        0.0       0.0   -2.0             2.0  
 2335582        0.0       

In [14]:
# Save pivot tables
import os
os.makedirs("artifacts/ab_test", exist_ok=True)

baseline_pivot.to_csv("artifacts/ab_test/model_advanced2.csv", index=False)

print("Saved model data for A/B testing:")
print("Model Advanced: artifacts/ab_test/model_advanced.csv")

Saved model data for A/B testing:
Model Advanced: artifacts/ab_test/model_advanced.csv


In [8]:
import os
pd.read_csv("artifacts/ab_test/model_advanced.csv").head()

,listing_id,aspect,date,negative,neutral,positive,score,total_mentions
0,2737,cleanliness_privacy,2015-04-22,0.0,0.0,1.0,1.0,1.0
1,2737,description_accuracy,2014-12-26,0.0,0.0,1.0,1.0,1.0
2,2737,nearby_restaurants_shops,2015-01-04,0.0,0.0,2.0,2.0,2.0
3,2737,nearby_restaurants_shops,2015-04-12,0.0,0.0,2.0,2.0,2.0
4,2737,return_intention,2014-12-26,0.0,0.0,1.0,1.0,1.0
